# Exploring Pyspark
Exploring Pyspark with NYC Yellow Taxi Data

In [ ]:
#!pip install pyspark

In [ ]:
#import pyspark

Pyspark will not run if Java is not installed on the computer. PySpark 4.x requires Java 17 or 21. PySpark 3.5 works perfectly with Java 8

In [ ]:
#pyspark.__version__

In [ ]:
#pip uninstall pyspark -y

In [ ]:
# Force install pyspark version
#!pip install pyspark==3.5.1

In [1]:
import pyspark

In [54]:
from pyspark.sql.functions import expr,col, regexp_replace, coalesce, lit, when
from pyspark.sql import SparkSession

In [2]:
#Build a Pyspark Session
spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [40]:
# read csv file in pyspark
df = spark.read.csv(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\csv_trip_data\yellow_tripdata_2024-01.csv'
    ,header=True #sets the first row to be the header
    ,inferSchema = True #Default is string if infer Schema is not specified.Ensures to infer the correct datatype of columns. 
)

In [ ]:
#Another way to read files
df=spark.read.option('header','true').csv(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\csv_trip_data\yellow_tripdata_2024-01.csv'
)

In [ ]:
df.show()

In [ ]:
# Check dataframe type
type(df)

In [ ]:
#Print Schema
df.printSchema()

In [ ]:
# Select columns in pyspark
df.select('VendorID','tpep_pickup_datetime').show()

In [45]:
#Read Parquet file
df_parq= spark.read.parquet(
    r'C:\Users\User\Documents\PORTFOLIO\NYC TAXI DATA\yellow_tripdata\yellow_tripdata_2024-01.parquet')
    

In [46]:
df_parq.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [ ]:
df_parq.printSchema()

In [50]:
# Select columns
df_parq.select('VendorID','tpep_pickup_datetime','store_and_fwd_flag').show(5)

+--------+--------------------+------------------+
|VendorID|tpep_pickup_datetime|store_and_fwd_flag|
+--------+--------------------+------------------+
|       2| 2024-01-01 00:57:55|                 N|
|       1| 2024-01-01 00:03:00|                 N|
|       1| 2024-01-01 00:17:06|                 N|
|       1| 2024-01-01 00:36:38|                 N|
|       1| 2024-01-01 00:46:51|                 N|
+--------+--------------------+------------------+
only showing top 5 rows



In [51]:
df_parq.select('store_and_fwd_flag').distinct().show(5)

+------------------+
|store_and_fwd_flag|
+------------------+
|                 Y|
|                 N|
|              NULL|
+------------------+



In [ ]:
df_parq.dtypes
df_parq.select('VendorID','tpep_pickup_datetime').dtypes

In [ ]:
df_parq.describe().show()

# Transformation Steps

In [ ]:
#Add column
# A simple add is df.withColumn('New Column Name',derived column_value).col references a dataframe column that is called
df_parq.withColumn("Vendor_name",
    when(col("VendorID") == 1, "Creative Mobile Technologies, LLC")
    .when(col("VendorID") == 2, "Curb Mobility, LLC")
    .when(col("VendorID") == 6, "Myle Technologies Inc")
    .when(col("VendorID") == 7, "Helix")
    .otherwise("No Vendor")
)\
.withColumn(
        "store_and_forward_trip_flag",
        coalesce(regexp_replace(col("store_and_fwd_flag"), '"', ''), lit("N/A"))
).show(1)

In [61]:
# Use sql case when inside Pyspark

## Add additional columns. To add multiple columns just continue adding .withColumn after each column. \ specifies a new line

df_parq = df_parq\
.withColumn(
    "vendor_name",
    expr("""
        case
            when vendorID = 1 then 'Creative Mobile Technologies, LLC'
            when VendorID = 2 then 'Curb Mobility, LLC'
            when VendorID = 6 then 'Myle Technologies Inc'
            when VendorID = 7 then 'Helix'
            else 'No Vendor'
        end
    """)
)\
.withColumn(
    "rate_code_type",
    expr("""
        case 
            when ratecodeid = 1 then 'Standard rate'
            when ratecodeid = 2 then 'JFK'
            when ratecodeid = 3 then 'Newark'
            when ratecodeid = 4 then 'Nassau or Westchester'
            when ratecodeid = 5 then 'Negotiated fare'
            when ratecodeid = 6 then 'Group ride'
            when ratecodeid = 99 then 'Unknown'
            else 'N/A'
        end
    """)
)\
.withColumn(
    "payment_type", #since column name already exists, it overwrites it
    expr("""
        case
            when payment_type = 0 then 'Flex Fare trip'
            when payment_type = 1 then 'Credit card'
            when payment_type = 2 then 'Cash'
            when payment_type = 3 then 'No charge'
            when payment_type = 4 then 'Dispute'
            when payment_type = 5 then 'Unknown'
            when payment_type = 6 then 'Voided trip'
            else 'N/A'
        end
    """)
)


In [62]:
#Add forgotten column
df_parq = df_parq\
.withColumn("trip_duration_mins", 
            expr("(unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 60")
)\
.withColumn("store_and_forward_trip_flag",
            expr("coalesce(regexp_replace(store_and_fwd_flag, '\"', ''), 'N/A')")
)

In [63]:
df_parq.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+---------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|         vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+-

In [64]:
df_parq.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: string (nullable = false)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- vendor_name: string (nullable = false)
 |-- rate_code_type: string (nullable = false)
 |-- trip_duration_mins: double 

In [66]:
##Drop Columns
df_parq=df_parq.drop('store_and_fwd_flag')

In [67]:
df_parq.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------+------------------+---------------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|       vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+--------+--------------------+---------------------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------+------------------+---------------------------+
|

In [ ]:
## Rename columns
#df_parq.withColumnRenamed('Airport_fee','airport_fee')

In [69]:
##Rename multiple columns
df_parq=df_parq\
    .withColumnRenamed('VendorID','vendor_id').withColumnRenamed('RatecodeID','rate_code_id')\
    .withColumnRenamed('tpep_pickup_datetime','pickup_time').withColumnRenamed('tpep_dropoff_datetime','dropoff_time')\
    .withColumnRenamed('DOLocationID','droppoff_zone').withColumnRenamed('PULocationID','pickup_zone')\
    .withColumnRenamed('Airport_fee','airport_fee').withColumnRenamed('extra','extra_fees')

In [70]:
df_parq.show(2)

+---------+-------------------+-------------------+---------------+-------------+------------+-----------+-------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+---------------------------+
|vendor_id|        pickup_time|       dropoff_time|passenger_count|trip_distance|rate_code_id|pickup_zone|droppoff_zone|payment_type|fare_amount|extra_fees|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|         vendor_name|rate_code_type|trip_duration_mins|store_and_forward_trip_flag|
+---------+-------------------+-------------------+---------------+-------------+------------+-----------+-------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+--------------+------------------+---------

In [73]:
# reorder columns
cols = [
        "vendor_id","vendor_name","pickup_time","dropoff_time","trip_duration_mins","passenger_count","trip_distance"
        ,"rate_code_id","rate_code_type","store_and_forward_trip_flag","pickup_zone","droppoff_zone","payment_type"
        ,"fare_amount","extra_fees","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount"
        ,"congestion_surcharge","airport_fee"
       ]
df_parq= df_parq.select(cols)

In [74]:
df_parq.show(3)

+---------+--------------------+-------------------+-------------------+------------------+---------------+-------------+------------+--------------+---------------------------+-----------+-------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------------+-----------+
|vendor_id|         vendor_name|        pickup_time|       dropoff_time|trip_duration_mins|passenger_count|trip_distance|rate_code_id|rate_code_type|store_and_forward_trip_flag|pickup_zone|droppoff_zone|payment_type|fare_amount|extra_fees|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+---------+--------------------+-------------------+-------------------+------------------+---------------+-------------+------------+--------------+---------------------------+-----------+-------------+------------+-----------+----------+-------+----------+------------+---------------------+------------+--------------